## Learning Automaton Forumscrape Project
# Data Analytics

This notebook explains how to derive some basic statistics from the forum post data we gathered from the [Fruits and Veggies forum](https://learningautomaton.ca/wp-content/uploads/2019/02/FruitsAndVeggiesForum/Knock%20Knock...%20-%20Fruits%20and%20Veggies.html) as part of my [Forumscrape Project](https://learningautomaton.ca/2019/01/ethical-forum-scraping-and-nlp-data-analytics-project/).

In the [previous post](https://learningautomaton.ca/2019/04/forum-scrape-project-data-processing-sanitation-and-anonymizing/) we cleaned and anonymized the data we scraped from the forum.

To make this notebook interactive I have embedded the post contents into a string and included it in the notebook.

In this notebook we will:
* Strip out unneeded words from our data
* Make a custom pipeline to label words as being fruit or vegetable words
* Count mentions of fruits and vegetables, and find words describing fruits or veggies to get an idea of which is preferred on the forum

Tools: To do the language processing and analytics, we will use [spaCy](https://spacy.io/), a free, open-source Natural Language Processing library for Python.


## A Primer on Pipelines, and Setup

I recommend you take a look at the [spaCy doc page on pipelines](https://spacy.io/usage/processing-pipelines). 

In short, when you pass a string or iterable to a spaCy language object (retrieved using the spacy.load(language model) call), it passes the string through a series of pipes that each do a specific thing, and stores the results in the returned `Doc` object. 
    
For example, a fairly standard (and default) pipeline is as follows:
* The `tokenizer` pipe breaks the string into tokens (words) which are stored as `token` objects in the `doc`
* The `tagger` pipe tries to identify what part of speech (pos) (noun, verb, adjective, etc) best describes each token, and adds it to that token's `pos` attribute
* The `parser` pipe adds dependency labels to various attributes
* The `ner` pipe tries to identify named entities and adds them to doc.ents
* etc...

Note that there are some site-specific *slang* words for fruits and veggies we need to watch out for. In this data, "apples" are often referred to as "pulls". If we didn't explicitly label this word, it would be harder to find as spaCy sometimes tags it as a verb and sometime as a noun, depending on context.

In [ ]:
# Setup
import spacy
# We use the en_core_web_sm language model as it is lightweight and suits our needs. For more complex datasets, a more robust model should be selected, such as en_core_web_md, en_core_web_lg, or any of the many available to download through spaCy
# Note that if you are running this in myBinder from my github, en_core_web_sm should be loaded onto the VM. Otherwise, refer to the spaCy docs on how to download language models.  
nlp = spacy.load('en_core_web_sm')
data = "Apples are the best. Us enlightened who call them, 'pulls love them more than any other fruit. 'Pulls are tasty, sweet, and crisp. All other fruits and those yucky vegetables are inferior. Discuss.. Ya, 'pulls are the best. 'Pulls are ripe, juicy, and taste sooooo good. Nothing else can match! I'd eat apples every day all day if I could, doctors be damned!. Yall get real. Apples are gross. The worst of the fruits, which are all bad. Everyone knows that veggies are powerfood. Get your weak apple sauce out of here.. Everyone knows grapes are superior you filthy casual. Enjoy your cyanide balls. Battlestar Galactica.. NO ONE READ THE ABOVE. THAT IS NOT ME.  Identity theft is not a joke, Jim! Millions of families suffer every year!. MICHAEL!  *storms off*. Oh, that's funny.  MICHAEL. From my head tomatoes.. Never Leaf Me.  We make a pearfect couple.. Peaches are better than all others. Peaches are the best, cabbage is the worst, end of story.  Smelly Cabbage; yuck. Corn cob rows!  Take me home,  to the placeeee,  I belonggg!!!!  WEST INDIANAAAAA  BUTTER MAMAAAAA  TAKE ME HOMEEEEE  CORN BOB ROWWWSS. If a squash can make you smile.... Lol, tomatoes and squash are both fruits! Get this compost out of the veggie master plate forum. Na, tomatoes and squash are fruits in name only. Acidic tomatoes and dirty squash, just like dirty tomatoes, should stay out of our sweet fruit forum. Like yucky lettuce, they belong in the vegetable forum, aka the compost.. Knock Knock. Who's There?. Banana. Banana who?. Knock Knock.... Who's There?... Banana. Banana who?. Knock Knock. Whos. ... There..... Orange!. Orange who?. ORANGE YOU GLAD I DIDN'T SAY BANANA??!?. You thought you'd pull a fast one on me, you little bitch? I'll have you know I graduated top of my class in the Tasty Peels, and I've been involved in numerous secret raids on Al-Quinoa, and I have over 300 confirmed spills. I am trained in vanilla warfare and I'm the top ripener in the entire US farmed forces. You are nothing to me but just another target grocery aisle. I will wipe you the fuck out with precision the likes of which has never been seen before on this Earth, mark my fucking words. You think you can get away with saying that shit to me over the Internet? Think again, fucker. As we speak I am contacting my secret network of pies across the USA and your IP is being traced right now so you better prepare for the storm, maggot. The storm that wipes out the pathetic little thing you call your life. You're fucking dead, kid. I can be anywhere, anytime, and I can eat you in over seven hundred ways, and that's just with my bare hands. Not only am I extensively trained in organic combat, but I have access to the entire pesticide arsenal of the United States Soybean Crops and I will use it to its full extent to wipe your miserable ass off the face of the continent, you little shit. If only you could have known what appleholey retribution your little \"clever\" comment was about to bring down upon you, maybe you would have held your fucking tongue. But you couldn't, you didn't, and now you're paying the price, you goddamn idiot. I will shit fury all over you and you will drown in it. Your lettuce head is fucking dead, kiddo.. Get boiled, damn."
fruit_match_words = set(['fruit','apple','apricot','banana','cantaloupe','cherry','date','citrus','avocado','carissa','guava','cherry','citron','clementine','crabapple','grape','grapefruit','honeydew','lemon','lime','orange','mandarin','mango','papaya','peach','pear','pineapple','plantain','plum','pomelo','tangarine','watermelon'])
vegetable_match_words =  set(['vegetable','artichoke','eggplant','asparagu','broccoli','cabbage','cauliflower','celery','spinach','lettuce','onion','beet','carrot','potato','yam','turnip','squash','tomato','watercress'])

## Removing Stop Words

Before carrying on, it is worth removing words that might interfere with our search for joining words. Removing stop words (words that are used in a language that don't do much to alter meaning for NLP), words we do not have in our vocabulary, or symbols and punctuation we don't need will make things simpler when it comes to determining words used to describe nouns.

Because we only have one slang word in our data set, we will manually replace it as well, instead of mucking about with vocabularies and custom mappings. We do this by looking for words with the lemma of the slang words that are used like a noun (determined by spaCy based on context and usage) and replace it with a new token for the word we want to replace.

(`pull` -> `apple`)

This is still pre-processing, as we haven't added our labelling pipe yet.

In [ ]:
# Remove stop tokens, parts of speech that aren't helpful for what we are looking for, and extra spaces. We keep periods so we can look at sentences.

keep_tokens = [token for token in nlp(data) if not( # Iterating over a spaCy Doc returns the tokens within the Doc 
        token.is_stop or # remove stop words, or words that aren't useful for most NLP problems
        (token.pos_ in ['PROPN','NUM','SYM','X','PUNCT'] and not token.text==".") or # remove proper nouns, numbers, symbols, unknowns, and all punctuation except periods, so we can delineate sentences.
        token.is_space # Remove blank sentences
    )]

# Find every instance of a word with the lemma `pull` that is used like a noun, and change it to apple.  
keep_tokens = list(map(lambda token: nlp('apple') if token.lemma_ == 'pull' and token.pos_ == 'NOUN' else token, keep_tokens))
print(keep_tokens)

## Adding the Labeling Pipe

Because we only care about tagging nouns as "FRUITS" or "VEGETABLES", we are going to disable the default `ner` pipe (to avoid clashing labels) and add our own `EntityMatcher` pipe, which will use a `PhraseMatcher` to determine what words to label.

First, we make a generic `EntityMatcher` that takes in a spaCy language object, the name of the matcher (which can be used to modify or disable the matcher later on), an iterable of terms we want to match, and a label to apply to those terms if we find them. 

We will also use a nifty feature added in spaCy v2.1 that allows us to choose what *attribute* of each token we want to match on (in this case, match on word lemmas, not the verbatim text (default). Because attributes are added by other pipes, we want our pipe to be last. We are going to match on the "LEMMA" attribute, which is the base form of the word (the lemma of "Apples" is "apple", the lemma of "going" is "go").

In [ ]:
# Make a generic EntityMatcher pipe
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

class EntityMatcher(object):
    # Takes in the spaCy language model initialized by spacy.load('en_core_web_sm'), the name of the matcher, 
    # a list of terms to match on, and the label to apply.
    def __init__(self, nlp, matcher_name, terms, label):
        # We run each term through the language model to make a doc object for each one, so we can access the "LEMMA" property of the
        # match terms (required to make use of the attr='LEMMA' feature, even though the terms are already in lemma form).
        patterns = [nlp(text) for text in terms]
        # The PhraseMatcher will find tokens in the doc that match the match terms 
        self.matcher = PhraseMatcher(nlp.vocab, attr='LEMMA')
        # Add our patterns (one for each FRUIT match word, for example) and the label to apply ("FRUIT")
        self.matcher.add(label, None, *patterns)
        # Set the name of the matcher
        self.name = matcher_name

    # When called on a Doc object
    def __call__(self, doc):
        # Get the list of matches from the matcher we created above
        matches = self.matcher(doc)
        # Make a Span (list of sequential tokens) for the matches and save them to the doc.ents list
        # (The matcher only returns the index of the first and last token of the match, so we need to make a Span that gets the words in between)
        for match_id, start, end in matches:
            span = Span(doc, start, end, label=match_id)
            doc.ents = list(doc.ents) + [span]
        # Return the Doc object, with our changes
        return doc

In [ ]:
# Make a new matcher for our list of fruit words and list of vegetable words
fruit_matcher = EntityMatcher(nlp, "fruit_matcher", fruit_match_words, "FRUIT")
vegetable_matcher = EntityMatcher(nlp, "vegetable_matcher", vegetable_match_words, "VEGETABLE")

# Disable the default ner pipe, which causes conflicts as it mis-labels some fruits/vegetables as named entities based on context.
nlp.disable_pipes("ner")
# Add our pipes to the language model.
# Note: spaCy will not let you add the same pipe twice. If you want to make changes to the pipe, you need to reload the model or use
# nlp.replace_pipe('nameOfOldPipe', newPipe), or nlp.remove_pipe('nameOfOldPipe'), nlp.add_pipe(newPipe)
nlp.add_pipe(fruit_matcher)
nlp.add_pipe(vegetable_matcher)

nlp.pipeline

Now we call our updated language with the data we want to be parsed, and store it as a Doc object. 

The labelled **entities** we found using our matchers were stored as `Spans` in the `doc.ents` variable. Let's see that text was labelled, the text lemma, and the tag.

We can also get each individual **token's** label by getting its `.ent_type_`. 

*Note: spaCy stores strings as hashes for speed. `.ent_type` is the hash string, and adding a trailing underscore tells spaCy to fetch the string, not the hash. You will notice this used for several variables.*

In [ ]:
doc = nlp(' '.join([token.text for token in keep_tokens]))

# Large output, will show every remaining token in the doc
# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.ent_type_)

In [ ]:
# Proving that fetching texts by token and from the ent list both work
print(
    [token.text for token in doc if token.ent_type_ in ['FRUIT','VEGETABLE']] == 
    [ent.text for ent in doc.ents if ent.label_ in ['FRUIT','VEGETABLE']]
)

In [ ]:
for token in [token for token in doc if token.ent_type_ in ['FRUIT','VEGETABLE']]:
    print(token.text, token.pos_)

Note that this isn't perfect. Because we are using the default dependency pipes, the language is not optimized for our dataset: notice that "squash" was incorrectly tagged as a verb. We could optimize with more complex rules, or if we had sufficient data we could train our model to better recognize context to differentiate between the two uses, however that is beyond the scope of this stage of the project. For now, we will accept data with 'pretty good' accuracy.

Now that the data is labelled, we can use these labels to find these words again without needing to re-process the data. 

## Simple Analytics

Now what we have our data cleaned and pre-processed, we can do some simple (or complex) analysis on the data. We can do simple counts of occurrence, for each tagged word and for a total count for each tag:

In [ ]:
# Count occurrences of fruits and vegetables in the text

from collections import Counter
fruits = Counter()
veggies = Counter()
for ent in doc.ents:
    (fruits if ent.label_ == "FRUIT" else veggies).update([ent.lemma_])
print(fruits)
print(veggies)

In [ ]:
# Or, to get a count of all ent types (or of any attribute)

from spacy.attrs import ENT_TYPE
counts = doc.count_by(ENT_TYPE)
for c in counts:
    print(nlp.vocab.strings[c],counts[c])

We can also do more complex pattern matching.

In this example, we make Matcher objects for two scenarios: A tagged word followed by an adjective/adverb, or an adjective/adverb followed by a tagged word.

Note that this is where stripping out the useless words comes into play. If we still had to deal with in-between words, we would need much more complex rules.

In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

noun_adj_adv = [{'ENT_TYPE': {'IN': ['FRUIT','VEGETABLE']}}, {'POS': {'IN': ['ADJ','ADV']}}]
adj_adv_noun = [{'POS': {'IN': ['ADJ','ADV']}},{'ENT_TYPE': {'IN': ['FRUIT','VEGETABLE']}}]

matcher.add('NounIsAdjOrAdv', None, noun_adj_adv)  # add pattern
matcher.add('AdjOrAdvIsNoun', None, adj_adv_noun)

matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

We can go one step further and apply a score to the fruits and vegetables based on the words used to describe them. There are lists of words and their sentiment scores on the internet, and tools that can assist. For our purposes, we will use the abbreviated ad-hoc list below. 

In [ ]:
from collections import defaultdict
from statistics import mean
# Example sentiment scores, roughly based on the list found at https://github.com/aesuli/SentiWordNet
sentiment_scores = {
    'best':0.75,
    'tasty':0.65,
    'yucky':-0.75,
    'inferior':-0.25,
    'ripe':0.3,
    'gross':-0.75,
    'good':0.5,
    'worst':0.75,
    'bad':-0.6,
    'weak':-0.3,
    'superior':0.6,
    'better':0.5,
    'acidic':-0.2,
    'dirty':-0.70,
    'sweet':0.25,
}

word_sentiment = defaultdict(float)

for match_id, start, end in matches:
    if doc[start].pos_ == 'NOUN':
        noun = doc[start]
        descriptor = doc[start+1]
    else:
        noun = doc[start+1]
        descriptor = doc[start]
    word_sentiment[noun.lemma_]+=sentiment_scores[descriptor.lemma_]

for item in sorted([(word, value) for word, value in word_sentiment.items()], key = lambda x: x[1], reverse = True):
    print(item)

The initial look shows that fruits received better than vegetables individually, but there is a lot of trash talking about fruits and vegetables in general

From here we could go further down by applying terms used to describe individual fruits to the general fruit label and vice versa, and factoring in the frequency of mentions in different ways, but let's leave it there for now

## Conclusion

In this post we have:
* Strip out unneeded words from our data
* Made a custom pipeline to label words as being fruits or vegetables
* Counted mentions of fruits and vegetables, and found words describing fruits or veggies to get an idea of which is preferred on the forum

This concludes the coding portion of the Fruits and Veggies Scraping and Data Analytics Project. In the final post, we will recap what was covered and learned along the way.

If you have any comments, questions, or recommendations for improvement, please leave a comment or reach out to me at LearningAutomaton.ca.

Thank you again for reading.

<pre><code>if can_learn: 
    learn()</code></pre>